In [1]:
import pandas as pd
from glasspy.predict import GlassNet
from glasspy.data import SciGlass
from functools import partial

from funcoes import populacao_cnb as cria_populacao
from funcoes import funcao_objetivo_pop_cnb as funcao_objetivo_pop
from funcoes import selecao_torneio_max as funcao_selecao
from funcoes import cruzamento_ponto_duplo as funcao_cruzamento
from funcoes import mutacao_sucessiva_cnb as funcao_mutacao

C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
NUM_COMPOUNDS = 80
VALOR_MAX_COMPOUNDS = 100

TAMANHO_POPULACAO = 300
NUM_GERACOES = 5
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.05
CHANCE_DE_MUTACAO_POR_GENE = 0.25
TAMANHO_TORNEIO = 3

In [13]:
source = SciGlass()
df = source.data

df_compounds_and_prices = pd.read_csv('../analise_exploratoria/Prices.csv',sep=',')
compounds_list  = list(df_compounds_and_prices['Oxide Coumpounds'])

model = GlassNet()

funcao_objetivo = partial(funcao_objetivo_pop, compostos=compounds_list, modelo=model)

C:\Users\jose23038\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
populacao = cria_populacao(TAMANHO_POPULACAO, NUM_COMPOUNDS, VALOR_MAX_COMPOUNDS)

hall_da_fama = []

for n in range(NUM_GERACOES):
    print(f'Geração {n}', end='\r')
    
    # Seleção
    fitness = funcao_objetivo(populacao)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao(proxima_geracao, CHANCE_DE_MUTACAO, CHANCE_DE_MUTACAO_POR_GENE, VALOR_MAX_COMPOUNDS)
    
    # Atualização do hall da fama
    fitness = funcao_objetivo(proxima_geracao)
        
    maior_fitness = max(fitness)
    indice = fitness.index(maior_fitness)
    hall_da_fama.append(proxima_geracao[indice])    
    
    # Encerramento
    populacao = proxima_geracao
    
    
fitness = funcao_objetivo(hall_da_fama)
maior_fitness = max(fitness)
indice = fitness.index(maior_fitness)
melhor_individuo_observado = hall_da_fama[indice]

melhor_individuo_observado

[12,
 84,
 55,
 25,
 14,
 20,
 49,
 22,
 7,
 63,
 40,
 42,
 69,
 73,
 55,
 46,
 16,
 20,
 54,
 17,
 58,
 0,
 16,
 94,
 15,
 14,
 38,
 48,
 20,
 52,
 56,
 56,
 40,
 89,
 41,
 92,
 21,
 95,
 11,
 75,
 47,
 0,
 69,
 70,
 35,
 65,
 14,
 20,
 79,
 7,
 50,
 82,
 89,
 85,
 80,
 72,
 31,
 82,
 64,
 72,
 35,
 39,
 58,
 33,
 19,
 92,
 14,
 59,
 92,
 54,
 97,
 22,
 22,
 31,
 0,
 53,
 97,
 4,
 97,
 61]

In [20]:
dict_composition = dict(zip(compounds_list, melhor_individuo_observado))
display(list(dict_composition.items()))
predictions = model.predict(dict_composition)
predictions

[('SiO2', 12),
 ('P2O5', 84),
 ('ZrO2', 55),
 ('Na2O', 25),
 ('Al2O3', 14),
 ('Fe2O3', 20),
 ('CaO', 49),
 ('MgO', 22),
 ('MnO', 7),
 ('GeO2', 63),
 ('Li2O', 40),
 ('Ta2O5', 42),
 ('ZnO', 69),
 ('SrO', 73),
 ('CdO', 55),
 ('SnO2', 46),
 ('B2O3', 16),
 ('La2O3', 20),
 ('Ga2O3', 54),
 ('Y2O3', 17),
 ('TiO2', 58),
 ('Nb2O5', 0),
 ('PbO', 16),
 ('HfO2', 94),
 ('WO3', 15),
 ('Sb2O3', 14),
 ('Bi2O3', 38),
 ('BaO', 48),
 ('Cr2O3', 20),
 ('Cu2O', 52),
 ('BeO', 56),
 ('CuO', 56),
 ('Nd2O3', 40),
 ('CeO2', 89),
 ('Cs2O', 41),
 ('As2O3', 92),
 ('NH4NO3', 21),
 ('MoO3', 95),
 ('FeO', 11),
 ('Mn2O3', 75),
 ('SO2', 47),
 ('Ag2O', 0),
 ('TeO2', 69),
 ('CoO', 70),
 ('In2O3', 35),
 ('Sc2O3', 65),
 ('NiO', 14),
 ('V2O5', 20),
 ('Li2SO4', 79),
 ('As2O5', 7),
 ('MnO2', 50),
 ('Sm2O3', 82),
 ('Gd2O3', 89),
 ('Tb2O3', 85),
 ('Dy2O3', 80),
 ('Ho2O3', 72),
 ('Er2O3', 31),
 ('Yb2O3', 82),
 ('Co3O4', 64),
 ('Fe3O4', 72),
 ('SnO', 35),
 ('Mn3O4', 39),
 ('Pr2O3', 58),
 ('CrO3', 33),
 ('Na2SO4', 19),
 ('Pr6O11', 9

,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,...,Cp1673K,TMaxGrowthVelocity,MaxGrowthVelocity,CrystallizationPeak,CrystallizationOnset,SurfaceTensionAboveTg,SurfaceTension1173K,SurfaceTension1473K,SurfaceTension1573K,SurfaceTension1673K
0,2216.480736,1456.782011,1147.175714,993.660589,1156.955013,1197.316122,898.240608,873.754869,875.335968,909.4125,...,1335.751014,978.934619,-4.176014,1437.154834,1384.461316,0.450866,0.3428,0.233737,0.33087,1.360293


In [17]:
float(predictions['Density293K'].iloc[0])

4.095341880866464